In [1]:
# First let's import our libraries
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import urllib.parse #TO PARSE PASSWORD FOR SQL CONNECTION

# To use dotenv
%load_ext dotenv
# To search for the dotenv file (.env) which contains the credentials
%dotenv

# Load in GA credentials from the dotenv file
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = %env GA_KEY
VIEW_ID = %env GA_VIEW_ID

# Set up GA connection
def initialize_analyticsreporting():
  
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    return analytics

# Query GA API
def get_report(analytics, date, metrics, dimension, filters=''):

    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                    'viewId': VIEW_ID,
                    'dateRanges': [date],
                    'metrics': metrics,
                    'dimensions': [{'name': dimension}],
                    'filtersExpression': filters,
                    'pageSize': 10000
                }]
        }
    ).execute()

# Parse the data and save it do a dataframe
def to_dataframe(response):
  
    # extract Data
    for report in response.get('reports', []):

        columnHeader = report.get('columnHeader', {})
        dimensionHeader = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])

        df_columns = dimensionHeader + [head['name'] for head in metricHeaders]
        df_rows = []
        
        for row in rows:
            dimension = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])[0].get('values', [])
            
            df_row = dimension + [int(value) for value in dateRangeValues]
            df_rows.append(df_row)
            
    return pd.DataFrame(df_rows, columns = df_columns)

In [ ]:
# To use sql magic
%load_ext sql

# Get CMS database credentials from .env file
db_password = %env DB_PASSWORD
db_user = %env DB_USER
db_host = %env DB_HOST
db_wp_database = %env DB_WP_DATABASE

# Converts sql password
password = urllib.parse.quote_plus(db_password)
# Connection string
connection =f'mysql+pymysql://{db_user}:{password}@{db_host}/{db_wp_database}'
# Connect, note, the dollar sign needs to be used
%sql $connection

# Query the WP database
sql_result = %sql SELECT post_date, post_name, post_title, post_type FROM wordpress.wp_posts WHERE post_status = 'publish';

# Convert into pandas dataframe
wp_df = sql_result.DataFrame()

In [9]:
date = {'startDate': '2020-01-01', 'endDate': 'today'}
metrics = [{'expression': 'ga:pageviews'}]
dimension = 'ga:pagePath'
filters = 'ga:pagePath!~preview'

analytics = initialize_analyticsreporting()
response = get_report(analytics, date, metrics, dimension, filters)
ga_df = to_dataframe(response)

In [13]:
def clean_pagepath(df):
    
    # add a new column containing the post_name extracted from the path
    df['post_name'] = df.apply(lambda row: urllib.parse.urlparse(row['ga:pagePath']).path.rsplit('/')[-2], axis = 1)
    
    # Rename empyt strings to home
    df['post_name'].replace('', 'home', inplace=True)
    
    # group post_names and sum metrics to get final count
    cleaned_df = df.groupby('post_name')['ga:pageviews'].sum()
    
    return cleaned_df

In [14]:
def merge_ga_wp(ga_df, wp_df):
    
    # merge dataframes
    merged_df = pd.merge(ga_df, wp_df, on='post_name', how='inner')
    
    return merged_df

In [ ]:
ga_cleaned_df = clean_pagepath(ga_df)
final_df = merge_ga_wp(ga_cleaned_df, wp_df)
final_df.sort_values('ga:pageviews', ascending=False).head()

In [21]:
filtered_df = final_df[final_df.post_date.dt.year == 2020] \
.sort_values('ga:pageviews', ascending=False)

In [ ]:
filtered_df.head(10)